In [1]:
from sosmetrics.metrics import BinaryCenterMetric, PD_FAMetric, BinaryCenterAveragePrecisionMetric, AUC_ROC_PRMetric, Precision_Recall_F1_IoUMetric
import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [2]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)


  0%|          | 0/21 [00:00<?, ?it/s]

# Binary Center Metric

In [3]:

# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = BinaryCenterMetric(
    dis_thr=[1, 10],
    conf_thr=0.5,
    dilate_kernel_size=[7,7],
    match_alg='hungarian',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()
Metric.reset()

Reading image_name=0020: 100%|██████████| 21/21 [00:00<00:00, 90.71it/s]


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
BinaryCenterMetric.update() took 0.00s each time.
+-----------+----+----+----+------------------+---------------+----------------+
| Threshold | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+-----------+----+----+----+------------------+---------------+----------------+
|    1.0    | 34 | 16 | 18 |     0.68000      |    0.65385    |    0.66667     |
|    2.0    | 40 | 10 | 12 |     0.80000      |    0.76923    |    0.78431     |
|    3.0    | 40 | 10 | 12 |     0.80000      |    0.76923    |    0.78431     |
|    4.0    | 42 | 8  | 10 |     0.84000      |    0.80769    |    0.82353     |
|    5.0    | 42 | 8  | 10 |     0.84000      |    0.80769    |    0.82353     |
|    6.0    | 42 | 8  | 10 |     0.84000      |    0.80769    |    0.82353     |
|    7.0    | 42 | 8  | 10 |     0.84000      |    0.80769    |    0.82353     |
|    8.0    | 42 | 8  |

# PD_FA Metric

In [4]:
# For test multiple format of input.
from sosmetrics.metrics import PD_FAMetric


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = PD_FAMetric(
    dis_thr=[1, 10],
    conf_thr=0.5,
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()
Metric.reset()

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PD_FAMetric.update() took 0.00s each time.
+-----------+----+----+-----+---------+-----------+-------------+
| Threshold | TD | AT |  FD |    NP   | target_Pd |   pixel_Fa  |
+-----------+----+----+-----+---------+-----------+-------------+
|    1.0    | 38 | 52 | 300 | 2752512 |  0.73077  | 1.08991e-04 |
|    2.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    3.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    4.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    5.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    6.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    7.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    8.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    9.0    | 44 | 52 | 108 | 2752512 |  0.84615  | 3.92369e-05 |
|    10.0   | 44 | 52 | 108 | 2752512 |  0.84615